In [60]:

import os

os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TRANSFORMERS_NO_JAX"] = "1"
os.environ["TRANSFORMERS_NO_PYTORCH_AUDIO"] = "1"
os.environ["TRANSFORMERS_NO_PYTORCH_VIS"] = "1"
os.environ["TRANSFORMERS_USE_PYTORCH"] = "1"

from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Dict, Any, List

import numpy as np
import pandas as pd
import re
import sqlite3
import pickle
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline

In [61]:
import sqlite3

def init_prompt_db(db_path="prompts.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prompts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE,
        category TEXT,
        description TEXT,
        template TEXT
    )
    """)

    prompt_entries = [
        (
            "avg_price",
            "analysis",
            "Professional price analysis",
            """
You are a senior Airbnb pricing analyst.

User Question: {question}
Data Summary:
{context}

Write a polished explanation including:
- Clear takeaway
- Bold highlights
- Price insights
- 3–5 concise sentences in professional tone.
"""
        ),
        (
            "cheapest_listings",
            "search",
            "Cheapest listing explanation",
            """
Summarize these listings with:
- Bullet points
- Bold price highlights
- Neighborhood strengths

User Question: {question}
Listings:
{context}
"""
        ),
        (
            "high_demand",
            "analysis",
            "High demand area explanation",
            """
User Question: {question}
Demand Data:
{context}

Explain:
- Why demand is high
- Which neighborhoods stand out
- What the data implies.

Format in 3–4 strong sentences.
"""
        ),
        (
            "presentation_mode",
            "presentation",
            "Short slide-ready explanation",
            """
### Key Insight
**{context}**

Produce a short presentation-ready summary (2–3 sentences).
Focus on clarity and bold highlights.
"""
        )
    ]

    cursor.executemany("""
        INSERT OR REPLACE INTO prompts (name, category, description, template)
        VALUES (?, ?, ?, ?)
    """, prompt_entries)

    conn.commit()
    conn.close()

    print("✔ Prompt DB initialized:", db_path)

# RUN THIS ONCE
init_prompt_db()


✔ Prompt DB initialized: prompts.db


In [62]:
class PromptDB:
    def __init__(self, db_path="prompts.db"):
        self.db_path = db_path

    def get(self, name):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT template FROM prompts WHERE name=?", (name,))
        row = cursor.fetchone()
        conn.close()
        return row[0] if row else None


In [63]:
DATA_PATH = Path("AB_NYC_2019.csv")

def load_dataset(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CSV not found at {path.resolve()}")

    df = pd.read_csv(path)

    required_cols = {
        "name",
        "neighbourhood_group",
        "neighbourhood",
        "room_type",
        "price",
        "number_of_reviews",
        "availability_365",
    }
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Dataset is missing required columns: {missing}")

    df = df.dropna(subset=["name", "neighbourhood_group", "neighbourhood"]).copy()
    df = df[df["price"] > 0]
    df = df.reset_index(drop=True)

    df["text"] = (
        df["name"].fillna("") + " " +
        df["neighbourhood_group"].fillna("") + " " +
        df["neighbourhood"].fillna("") + " " +
        df["room_type"].fillna("")
    ).str.lower()

    print(f"[INFO] Loaded {len(df)} listings after cleaning.")
    
    return df


In [64]:
df = load_dataset(DATA_PATH)
df.head()

[INFO] Loaded 48868 listings after cleaning.


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,text
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,clean & quiet apt home by the park brooklyn ke...
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,skylit midtown castle manhattan midtown entire...
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,the village of harlem....new york ! manhattan ...
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,cozy entire floor of brownstone brooklyn clint...
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,entire apt: spacious studio/loft by central pa...


In [65]:
class TfidfSearch:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.vectorizer = TfidfVectorizer(stop_words="english")
        self.matrix = self.vectorizer.fit_transform(df["text"].fillna(""))

    def search(self, query: str, k: int = 10, candidate_idx=None):
        if candidate_idx is None:
            sub_matrix = self.matrix
            base_df = self.df
        else:
            valid_idx = candidate_idx[candidate_idx < self.matrix.shape[0]]
            sub_matrix = self.matrix[valid_idx]
            base_df = self.df.iloc[valid_idx]


        q_vec = self.vectorizer.transform([query.lower()])
        sims = cosine_similarity(q_vec, sub_matrix)[0]

        order = np.argsort(sims)[::-1][:k]
        result = base_df.iloc[order].copy()
        result["similarity"] = sims[order]
        return result


In [66]:
BOROUGH_ALIASES = {
    "manhattan": "Manhattan",
    "brooklyn": "Brooklyn",
    "queens": "Queens",
    "bronx": "Bronx",
    "staten island": "Staten Island",
    "staten": "Staten Island",
}

from dataclasses import dataclass
from typing import Optional
import re


@dataclass
class QuerySpec:
    raw_text: str
    task: str
    borough: Optional[str] = None
    cheapest: bool = False
    most_expensive: bool = False
    avg_price: bool = False
    high_reviews_low_availability: bool = False
    max_price: Optional[float] = None
    k: int = 5


def _extract_all_numbers(text: str):
    """Extract all numbers, ignoring currency symbols."""
    cleaned = re.sub(r"[$,]", "", text)
    nums = []
    for token in cleaned.split():
        try:
            nums.append(float(token))
        except:
            pass
    return nums


def parse_query(text: str) -> QuerySpec:
    # Normalize input
    lower = text.lower()
    lower = re.sub(r"[·•\u2022\u00B7]", " ", lower)  # weird unicode dots
    lower = lower.replace("—", "-")
    lower = re.sub(r"\s+", " ", lower).strip()

    # Detect borough
    borough = None
    for key, canon in BOROUGH_ALIASES.items():
        if key in lower:
            borough = canon
            break

    # Detect top-k (e.g., find 10)
    k = 5  # default
    m = re.search(r"(?:find|top|show|give me|recommend)\s+(\d+)", lower)
    if m:
        k = int(m.group(1))

    # Cheapest + most expensive
    cheapest = any(w in lower for w in ["cheapest", "lowest price", "low price", "least expensive"])
    most_expensive = any(w in lower for w in ["most expensive", "highest price"])

    # Stats queries
    avg_price = "average price" in lower or "avg price" in lower or "mean price" in lower
    high_reviews_low_availability = (
        ("high reviews" in lower and "low availability" in lower)
        or "high demand" in lower
    )

    # Max price detection
    max_price = None

    # strong patterns: under/below/less than
    m_price = re.search(r"(?:under|below|less than)\s*\$?\s*(\d+)", lower)
    if m_price:
        max_price = float(m_price.group(1))

    # detect all other numbers
    nums = _extract_all_numbers(lower)

    for num in nums:
        # skip k numbers 
        if num == k:
            continue
        # Threshold to keep from misreading small numbers as prices
        if num >= 50:
            if not max_price:
                max_price = float(num)

    # Task type
    if avg_price or high_reviews_low_availability:
        task = "stats"
    else:
        task = "search"

    # Return parsed specification
    return QuerySpec(
        raw_text=text,
        task=task,
        borough=borough,
        cheapest=cheapest,
        most_expensive=most_expensive,
        avg_price=avg_price,
        high_reviews_low_availability=high_reviews_low_availability,
        max_price=max_price,
        k=k,
    )


In [67]:
# Test the parser
parse_query("find 10 cheapest rooms in Manhattan under $150")

QuerySpec(raw_text='find 10 cheapest rooms in Manhattan under $150', task='search', borough='Manhattan', cheapest=True, most_expensive=False, avg_price=False, high_reviews_low_availability=False, max_price=150.0, k=10)

In [68]:
def filter_by_spec(df: pd.DataFrame, spec: QuerySpec):
    sub = df
    if spec.borough:
        sub = sub[sub["neighbourhood_group"] == spec.borough]

    if spec.max_price:
        sub = sub[sub["price"] <= spec.max_price]

    return sub.copy()

def summarize_average_price(df: pd.DataFrame):
    if df.empty:
        return {"avg_price": None, "count": 0}
    
    return {
        "avg_price": float(df["price"].mean()),
        "median_price": float(df["price"].median()),
        "count": len(df),
    }

def find_cheapest(df: pd.DataFrame, k: int):
    return df.sort_values("price", ascending=True).head(k).copy()

def find_most_expensive(df: pd.DataFrame, k: int):
    return df.sort_values("price", ascending=False).head(k).copy()

def find_high_demand_neighbourhoods(df: pd.DataFrame, k: int = 5):
    grouped = (
        df.groupby(["neighbourhood_group", "neighbourhood"])
        .agg(
            avg_price=("price", "mean"),
            reviews=("number_of_reviews", "mean"),
            availability=("availability_365", "mean"),
            count=("id", "count"),
        )
        .reset_index()
    )
    grouped["demand_score"] = grouped["reviews"] / (grouped["availability"] + 1)
    return grouped.sort_values("demand_score", ascending=False).head(k)

In [69]:
class LanguageModel:
    def __init__(self, model_name="google/flan-t5-base"):
        self.pipe = None
        if pipeline is None:
            print("[WARN] transformers not available. Using fallback mode.")
        else:
            try:
                self.pipe = pipeline("text2text-generation", model=model_name)
            except Exception as e:
                print(f"[WARN] Model load failed: {e}")
                self.pipe = None

    def generate(self, prompt: str, max_new_tokens=128):
        if self.pipe is None:
            return "[No LLM available] " + prompt
        out = self.pipe(prompt, max_length=max_new_tokens, num_beams=4)[0]["generated_text"]
        return out.strip()

In [70]:
class AirbnbAgent:
    def __init__(self, df):
        self.df = df
        self.search_index = TfidfSearch(df)
        self.lm = LanguageModel()
        self.promptdb = PromptDB()  


    def _format_rows(self, rows, max_rows=10):
        lines = []
        for _, r in rows.head(max_rows).iterrows():
            lines.append(
                f"- {r['neighbourhood_group']} / {r['neighbourhood']} | "
                f"{r['room_type']} | ${r['price']} | "
                f"reviews={r['number_of_reviews']} | avail={r['availability_365']}"
            )
        return "\n".join(lines) or "No listings matched."

    def answer(self, question: str) -> str:
        clean_q = question
        clean_q = re.sub(r"[·•\u2022\u00B7]", " ", clean_q)  
        clean_q = clean_q.replace("—", "-")                 
        clean_q = re.sub(r"\s+", " ", clean_q).strip()     

        clean_q
        
        spec = parse_query(clean_q)
        sub = filter_by_spec(self.df, spec)

        if sub.empty:
            return "No listings matched your query."
    
        # Stats: average price
        if spec.avg_price:
            stats = summarize_average_price(sub)

            context = (
                f"Average price ≈ ${stats['avg_price']:.0f}, "
                f"median = ${stats['median_price']:.0f}, "
                f"based on {stats['count']} listings."
            )

            template = self.promptdb.get("avg_price")   # <-- SQL prompt
            prompt = template.format(question=clean_q, context=context)
            return self.lm.generate(prompt)

        # High demand areas (reviews high, availability low)
        if spec.high_reviews_low_availability:
            demand = find_high_demand_neighbourhoods(sub, spec.k)
            context = demand.to_string(index=False)

            template = self.promptdb.get("high_demand")   # <-- SQL prompt
            prompt = template.format(question=clean_q, context=context)
            return self.lm.generate(prompt)

        # Search queries + ranking
        candidate_idx = sub.index.to_numpy()
        ranked = self.search_index.search(clean_q, k=spec.k, candidate_idx=candidate_idx)

        if spec.cheapest:
           ranked = find_cheapest(sub, spec.k)
           prompt_name = "cheapest_listings"
        elif spec.most_expensive:
            ranked = find_most_expensive(sub, spec.k)
            prompt_name = "cheapest_listings"  # same template works well
        else:
            prompt_name = "cheapest_listings"
        
        context = self._format_rows(ranked, spec.k)

    # Load SQL template
        template = self.promptdb.get(prompt_name)
        prompt = template.format(question=clean_q, context=context)
        return self.lm.generate(prompt)


In [71]:
agent = AirbnbAgent(df)

questions = [
    "Average price in Brooklyn",
    "find 5 cheapest rooms in Manhattan under $120",
    "areas with high reviews and low availability in Queens",
]

for q in questions:
    print("="*80)
    print("Q:", q)
    print("A:", agent.answer(q))

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: Average price in Brooklyn
A: The average price in Brooklyn is $124, the median is $90, based on 20089 listings.
Q: find 5 cheapest rooms in Manhattan under $120


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A: Manhattan / SoHo | Private room | $10 | reviews=2 | avail=0 - Manhattan / Upper East Side | Entire home/apt | $10 | reviews=10 | avail=0 - Manhattan / Harlem | Shared room | $10 | reviews=2 | avail=0 - Manhattan / Hell's Kitchen | Shared room | $10 | reviews=2 | avail=0 - Manhattan / SoHo | Private room | $10 | reviews=2 | avail=0 - Manhattan / SoHo | Entire home/apt | $10 | reviews=10 | avail=0 - Manhattan / Harlem | Shared room | $10 | reviews=2 | avail=0 - Manhattan / SoHo | Private room | $10 | reviews=2 | avail=0 - Manhattan / SoHo | Entire home/apt | $10 | reviews=10 | avail=0 - Manhattan / Harlem | Shared room | $10 | reviews=2 | avail=0 - Manhattan / Hell's Kitchen | Shared room | $10 | reviews=2 | avail=0 - Manhattan / SoHo | Private room | $10 | reviews=2
Q: areas with high reviews and low availability in Queens


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A: Queens East Elmhurst 81.183784 81.659459 169.491892 185 0.478964 Queens Springfield Gardens 94.235294 69.094118 204.011765 85 0.337025 Queens Fresh Meadows 99.500000 32.656250 137.968750 32 0.234990 Queens South Ozone Park 82.400000 48.675000 226.100000 40 0.214333


In [72]:
question_1 = "suggest most affordable neighborhoods in Brooklyn"
parse_query(question_1)

QuerySpec(raw_text='suggest most affordable neighborhoods in Brooklyn', task='search', borough='Brooklyn', cheapest=False, most_expensive=False, avg_price=False, high_reviews_low_availability=False, max_price=None, k=5)

In [73]:
question = "suggest most affordable neighborhoods in Brooklyn"
print("Q:", question)
print("A:", agent.answer(question))

Q: suggest most affordable neighborhoods in Brooklyn


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A: Brooklyn / Bedford-Stuyvesant | Entire home/apt | $105 | reviews=12 | avail=0 - Brooklyn / Bushwick | Private room | $38 | reviews=48 | avail=3 - Brooklyn / Bushwick | Private room | $38 | reviews=33 | avail=1 - Brooklyn / Bedford-Stuyvesant | Private room | $53 | reviews=52 | avail=1


In [74]:
question_1 = "find 5 best rooms in Manhattan under $2000"

print("Q:", question_1)
print("A:", agent.answer(question_1))

Q: find 5 best rooms in Manhattan under $2000


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A: Manhattan / Upper West Side | Entire home/apt | $390 | reviews=108 | avail=156 - Manhattan / Washington Heights | Entire home/apt | $100 | reviews=1 | avail=0 - Manhattan / East Village | Entire home/apt | $400 | reviews=0 | avail=0 - Manhattan / Chinatown | Entire home/apt | $785 | reviews=85 | avail=331 - Manhattan / Harlem | Private room | $65 | reviews=0 | avail=362
